In [43]:
import lxml.html as lh
import time
import urllib.request
import argparse
import datetime
import pytz
from selenium import webdriver
from selenium.common.exceptions import StaleElementReferenceException
import os
#import coloredlogs


In [44]:
from docopt import docopt
from trading_bot.ops import get_state
from trading_bot.agent import Agent
from trading_bot.methods import evaluate_model
from trading_bot.utils import (
    get_stock_data,
    format_currency,
    format_position,
    show_eval_result,
    switch_k_backend_device
)

In [45]:
tz = pytz.timezone('Asia/Kolkata')

path1 = os.getcwd()
path = path1 + '/chromedriver.exe'
ignored_exceptions=(StaleElementReferenceException,)
options = webdriver.ChromeOptions()
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--headless')
options.add_experimental_option('excludeSwitches', ['enable-logging'])
#options.add_argument(f'user-agent={userAgent}')
driver = webdriver.Chrome(executable_path=path , options=options)


In [46]:
def Real(url,count):
    if count == 0:
        
        driver.get(url)
        #print(driver)

    else:
        driver.refresh()
        time.sleep(15)

    infile = driver.page_source
    doc = lh.fromstring(infile)
    live = doc.xpath('/html/body/div[1]/div/div/div[1]/div/div[2]/div/div/div[4]/div/div/div/div[3]/div/div/span[1]')
    live = float(live[0].text.replace(',',''))
    return live 

def main(args):
    count = 0
    total_profit = 0
    t=0
    history = []
    reward = 0
    ticker = args + '.NS'
    price = []
    window_size =10
    time_now = datetime.datetime.now(tz).time()
    while(datetime.time(9, 14, tzinfo=tz) < time_now < datetime.time(15, 31, tzinfo=tz)):
        url = 'https://finance.yahoo.com/quote/{}?p={}&.tsrc=fin-srch'.format(ticker,ticker)
        print(count)
        live = Real(url,count)
        count+=1        
        price.append(live)
        if count < window_size:
           continue
        model_name='model_debug_50'  
        print(live)
        initial_offset = price[1] - price[0]
        state = get_state(price, 0, window_size + 1)
        next_state = get_state(price, t + 1, window_size + 1)
        agent = Agent(state_size=window_size, pretrained=True, model_name=model_name)
        agent.inventory = []
        profit = evaluate_model(agent,state,next_state, price, t, total_profit, history, reward, window_size=window_size)
        show_eval_result(model_name, profit, initial_offset)
        t+=1
        state = next_state
    print("JAI MAHAKAL")
def evaluate_model(agent, state, next_state, data, t, total_profit, history, reward, window_size, debug=False):
  
    print(t)
        # select an action
    action = agent.act(state, is_eval=True)

        # BUY
    if action == 1:
        agent.inventory.append(data[t])

        history.append((data[t], "BUY"))
        if debug:
            logging.debug("Buy at: {}".format(format_currency(data[t])))

        # SELL
    elif action == 2 and len(agent.inventory) > 0:
        bought_price = agent.inventory.pop(0)
        delta = data[t] - bought_price
        reward = delta #max(delta, 0)
        total_profit += delta

        history.append((data[t], "SELL"))
        if debug:
            logging.debug("Sell at: {} | Position: {}".format(
                    format_currency(data[t]), format_position(data[t] - bought_price)))
        # HOLD
    else:
        history.append((data[t], "HOLD"))

#        done = (t == data_length - 1)
    agent.memory.append((state, action, reward, next_state))

    return total_profit

if __name__ == "__main__":
    #parser = argparse.ArgumentParser()
    #parser.add_argument('ticker', help = 'ticker')
    #args = parser.parse_args()
    args= input()
    #coloredlogs.install(level="DEBUG")
    switch_k_backend_device()

    try:
        main(args)
    except KeyboardInterrupt:
        print("Aborted")

 RELIANCE


0
1
2
3
4
5
6
7
8
9
1445.2
0
10
1443.55
1
11
1442.1
2
12
1442.0
3
13
1443.5
4
14
1443.15
5
15
1443.0
6
16
1443.75
7
17
1443.9
8
18
1444.2
9
19
1441.8
10
20
1442.05
11
21
1443.9
12
22
1443.75
13
23
1443.2
14
24
1444.0
15
25
1442.3
16
26
1442.5
17
27
1443.5
18
28
1442.2
19
29
1442.1
20
30
1441.95
21
31
1442.2
22
32
1442.4
23
33
1439.8
24
34
1439.55
25
35
1440.0
26
36
1440.8
27
37
1440.6
28
38
1442.15
29
39
1443.65
30
40
1444.9
31
41
1443.0
32
42
1443.3
33
43
1443.95
34
44
1443.05
35
45
1443.75
36
46
1443.65
37
47
1443.5
38
48
1440.9
39
49
1441.5
40
50
1440.75
41
51
1440.35
42
52
1440.7
43
53
1440.7
44
54
1440.9
45
55
1441.9
46
56
1441.65
47
57
1440.4
48
58
1441.0
49
59
1440.0
50
60
1441.5
51
61
1442.8
52
62
Aborted


In [47]:
import pandas as pd
import numpy as np

import altair as alt
import seaborn as sns

In [53]:
from trading_bot.agent import Agent

model_name = 'model_dqn_GOOG_50'
test_stock = 'data/GOOG_2019.csv'
window_size = 10
debug = True

agent = Agent(window_size, pretrained=True, model_name=model_name)

In [54]:
# read csv into dataframe
df = pd.read_csv(test_stock)
# filter out the desired features
df = df[['Date', 'Adj Close']]
# rename feature column names
df = df.rename(columns={'Adj Close': 'actual', 'Date': 'date'})
# convert dates from object to DateTime type
dates = df['date']
dates = pd.to_datetime(dates, infer_datetime_format=True)
df['date'] = dates

df.head()

,date,actual
0,2019-01-02,1045.849976
1,2019-01-03,1016.059998
2,2019-01-04,1070.709961
3,2019-01-07,1068.390015
4,2019-01-08,1076.280029


In [55]:
import logging
#import coloredlogs

from trading_bot.utils import show_eval_result, switch_k_backend_device, get_stock_data
from trading_bot.methods import evaluate_model

#coloredlogs.install(level='DEBUG')
switch_k_backend_device()

test_data = get_stock_data(test_stock)
initial_offset = test_data[1] - test_data[0]

test_result, history = evaluate_model(agent, test_data, window_size, debug)
show_eval_result(model_name, test_result, initial_offset)

[[0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5]]


In [56]:
def visualize(df, history, title="trading session"):
    # add history to dataframe
    position = [history[0][0]] + [x[0] for x in history]
    actions = ['HOLD'] + [x[1] for x in history]
    df['position'] = position
    df['action'] = actions
    
    # specify y-axis scale for stock prices
    scale = alt.Scale(domain=(min(min(df['actual']), min(df['position'])) - 50, max(max(df['actual']), max(df['position'])) + 50), clamp=True)
    
    # plot a line chart for stock positions
    actual = alt.Chart(df).mark_line(
        color='green',
        opacity=0.5
    ).encode(
        x='date:T',
        y=alt.Y('position', axis=alt.Axis(format='$.2f', title='Price'), scale=scale)
    ).interactive(
        bind_y=False
    )
    
    # plot the BUY and SELL actions as points
    points = alt.Chart(df).transform_filter(
        alt.datum.action != 'HOLD'
    ).mark_point(
        filled=True
    ).encode(
        x=alt.X('date:T', axis=alt.Axis(title='Date')),
        y=alt.Y('position', axis=alt.Axis(format='$.2f', title='Price'), scale=scale),
        color='action'
    ).interactive(bind_y=False)

    # merge the two charts
    chart = alt.layer(actual, points, title=title).properties(height=300, width=1000)
    
    return chart

In [57]:
chart = visualize(df, history, title=test_stock)
chart

alt.LayerChart(...)